In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [3]:
# Numpy のアップグレード
!pip install numpy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


ランタイムを再起動してください

In [1]:
# 環境変数を設定
!echo 'export LD_LIBRARY_PATH=/usr/lib' >> ~/.bashrc
!source ~/.bashrc

In [2]:
import pandas_datareader.data as pdr

def get_stock_data(code):
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [4]:
import plotly.graph_objs as go
import talib as ta
import datetime as dt
import pandas as pd
import numpy as np

name = '積水ハウス'
code = 1928
df = get_stock_data(code)

layout = { "height": 600,
           "title"  : { "text": "{} {}".format(code, name), "x":0.5 }, 
           "xaxis" : { "rangeslider": { "visible": False } },
           "yaxis": { "title": "価格（円）", "side": "left", 
                     "tickformat": "," },
          "plot_bgcolor": "light blue" }

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

# ローソク足
data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray")]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], 
                        x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

丸坊主

In [5]:
marubozu = ta.CDLMARUBOZU(df["Open"], df["High"], df["Low"], df["Close"])
# チャートに表示するテキスト
df["marubozu_text"] = marubozu.replace({ 100: "買い", -100: "売り", 0: "" })

# チャートに表示するマーカーの位置を算出
df["marubozu_marker"] = (marubozu/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["marubozu_marker"], 
                       mode="markers+text", 
                       text=rdf["marubozu_text"],
                       textposition="top center", 
                       name="丸坊主",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

寄付坊主 / 大引坊主

In [6]:
belthold = ta.CDLBELTHOLD(df["Open"], df["High"], df["Low"], df["Close"])
df["belthold_text"] = belthold.replace({ 100: "買い", -100: "売り", 0: "" })
df["belthold_marker"] = (belthold/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["belthold_marker"], 
                       mode="markers+text", 
                       text=rdf["belthold_text"],
                       textposition="top center", 
                       name="寄付坊主 / 大引坊主",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

包み足

In [7]:
engulfing = ta.CDLENGULFING(df["Open"], df["High"], df["Low"], df["Close"])
df["engulfing_text"] = engulfing.replace({ 100: "買い", -100: "売り", 0: "" })
df["engulfing_marker"] = (engulfing/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["engulfing_marker"], 
                       mode="markers+text", 
                       text=rdf["engulfing_text"],
                       textposition="top center", 
                       name="包み足",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

はらみ足

In [8]:
harami = ta.CDLHARAMI(df["Open"], df["High"], df["Low"], df["Close"])
df["harami_text"] = harami.replace({ 100: "買い", -100: "売り", 0: "" })
df["harami_marker"] = (harami/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["harami_marker"], 
                       mode="markers+text", 
                       text=rdf["harami_text"],
                       textposition="top center", 
                       name="はらみ足",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

包み上げ / 包み下げ

In [9]:
outside = ta.CDL3OUTSIDE(df["Open"], df["High"], df["Low"], df["Close"])
df["outside_text"] = outside.replace({ 100: "買い", -100: "売り", 0: "" })
df["outside_marker"] = (outside/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["outside_marker"], 
                       mode="markers+text", 
                       text=rdf["outside_text"],
                       textposition="top center", 
                       name="包み上げ / 包み下げ",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

はらみ上げ / はらみ下げ

In [10]:
inside = ta.CDL3INSIDE(df["Open"], df["High"], df["Low"], df["Close"])
df["inside_text"] = inside.replace({ 100: "買い", -100: "売り", 0: "" })
df["inside_marker"] = (inside * df["High"] /100).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["inside_marker"], 
                       mode="markers+text", 
                       text=rdf["inside_text"],
                       textposition="top center", 
                       name="はらみ上げ / はらみ下げ",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()